In [ ]:
## reference https://huggingface.co/learn/nlp-course/en/chapter7/6?fw=pt#training-a-causal-language-model-from-scratch

In [2]:
#installing some libraries
#!pip install datasets
!pip install --upgrade jupyter ipywidgets



Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch, transformers
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import PreTrainedModel, PretrainedConfig
from transformers import AutoModel, AutoConfig,AutoModelForCausalLM, AutoConfig,GPT2Config,GPT2Tokenizer
import torch.nn as nn
import torch.nn.functional as F
#from datasets import load_dataset
import pandas as pd, numpy as np
from torch.cuda.amp import autocast
from torch import cuda
import datetime
import warnings,itertools
from torch.optim.lr_scheduler import LambdaLR
import json
# Ignore all warnings
warnings.filterwarnings('ignore')
#pip install transformers bitsandbytes>=0.39.0 -q
import zipfile,logging

In [4]:
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
import random
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
#global params for training

B,T = 64,1024
epoch = 100
random_init_wts = True
min_text_len = 0
# hard coded com
comp_ratio = 3
# train_loss_list = []
# val_loss_list =[]
if cuda.is_available():
    device = torch.device('cuda:0')
    print(device)
else:
    device = 'cpu'
#print(device)
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
#os.environ["MKL_DEBUG_CPU_TYPE"] = "5"

#print(global_tr_loss)
model_path = os.path.join("model")
print(model_path)

cuda:0
model


In [6]:
directory = os.path.join('.','data','unzip_text_10M')  # Replace with your directory path
directory

'./data/unzip_text_10M'

In [7]:
def read_text(directory):
    directory = os.path.join('.','data','unzip_text_10M',str(directory))  # Replace with your directory path
    print(f"directory :{directory}")
    # List all files in the directory
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    print(f"files:{files}")
    text_content = []
    # Read each file
    total_lines = 0
    for filenum,filename in enumerate(files):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            #first_line = file.read()
            #print(f"filename :{filename}->first few lines {first_line}")
            #continue
            #lines_list = [line.strip() for line in open(file_path, 'r')]
            text = file.read()
            text_content.append(text)
            print(f"the file:{filename} has been appeneded to the uber list and its length is {len(text_content)} ")
            #total_lines+=len(lines_list)
            #text_content.append(lines_list)
    
    flattened_list = ''.join(text_content)
    assert (len(flattened_list) == total_lines , f"Expected {len(flattened_list)} to be equal to {total_lines}" )
    
    return flattened_list

    

In [8]:
train_list = read_text("train_10M")
#print(train_dict)
#val_list = read_text("dev")

directory :./data/unzip_text_10M/train_10M
files:['switchboard.train', 'simple_wiki.train', 'open_subtitles.train', 'gutenberg.train', 'childes.train', 'bnc_spoken.train']
the file:switchboard.train has been appeneded to the uber list and its length is 1 
the file:simple_wiki.train has been appeneded to the uber list and its length is 2 
the file:open_subtitles.train has been appeneded to the uber list and its length is 3 
the file:gutenberg.train has been appeneded to the uber list and its length is 4 
the file:childes.train has been appeneded to the uber list and its length is 5 
the file:bnc_spoken.train has been appeneded to the uber list and its length is 6 


In [9]:
len(train_list)

54215049

In [10]:
chunks = len(train_list)//(B*T)
print(chunks)

827


In [11]:
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})


In [12]:
print(tokenizer.pad_token_id)

50257


In [13]:
val_list = read_text("dev")


directory :./data/unzip_text_10M/dev
files:['switchboard.dev', 'simple_wiki.dev', 'open_subtitles.dev', 'gutenberg.dev', 'childes.dev', 'bnc_spoken.dev']
the file:switchboard.dev has been appeneded to the uber list and its length is 1 
the file:simple_wiki.dev has been appeneded to the uber list and its length is 2 
the file:open_subtitles.dev has been appeneded to the uber list and its length is 3 
the file:gutenberg.dev has been appeneded to the uber list and its length is 4 
the file:childes.dev has been appeneded to the uber list and its length is 5 
the file:bnc_spoken.dev has been appeneded to the uber list and its length is 6 


In [14]:
print(tokenizer.eos_token_id)

50256


In [15]:
random.random()

0.4800672319849466

In [16]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


In [17]:
def get_ranked_synonyms(word, pos):
    synonyms = []
    for syn in wordnet.synsets(word, pos=pos):
        for lemma in syn.lemmas():
            if lemma.name() != word:
                synonyms.append((lemma.name(), syn.wup_similarity(syn)))
    
    # Sort synonyms by similarity score in descending order
    ranked_synonyms = sorted(set(synonyms), key=lambda x: x[1] if x[1] is not None else 0, reverse=True)
    return [syn for syn, _ in ranked_synonyms]

In [18]:
lemmatizer = WordNetLemmatizer()

lemmatizer = WordNetLemmatizer()

def replace_adj_with_synonyms(text):
    words = word_tokenize(text)
    tagged = pos_tag(words)
    result = []
    for word, pos in tagged:
        if pos.startswith('J'):
            wordnet_pos = get_wordnet_pos(pos)
            lemma = lemmatizer.lemmatize(word, wordnet_pos)
            synonyms = get_ranked_synonyms(lemma, wordnet_pos)
            if synonyms:
                replacement = random.choice(synonyms)  # Choose from top 3 synonyms
                #print(f"word = {word}|replacement = {replacement}")
                result.append(replacement)
            else:
                result.append(word)
        else:
            result.append(word)
    return " ".join(result)


In [19]:
# sample_text = train_list[147888:152221]
# repl = replace_verbs_with_synonyms(sample_text)

In [20]:
# print(repl)


In [21]:
# print(sample_text)

In [22]:
def pad_sequences(token_ids_list, max_length = B*T, tokenizer = tokenizer):
    padded_sequences = tokenizer.pad(
        {"input_ids": token_ids_list},
        padding='max_length',
        max_length=max_length,
        return_tensors="pt"
    )
    return padded_sequences

In [23]:
def tokenize_text (text,tokenizer = tokenizer,max_length = B*T):
    #print(f"inside tokenize_text")
    enc = tokenizer(text,padding='max_length',truncation=True,max_length=max_length,return_tensors="pt",return_attention_mask=True)
    input_id = enc['input_ids']
    att_mask = enc['attention_mask']
    
    # now concatenate these lists to B*T
    input_id = torch.squeeze(input_id, dim = 0).to(dtype = torch.long)
    att_mask = torch.squeeze(att_mask, dim = 0).to(dtype = torch.bool)

    return input_id,att_mask
    
    
    
        
    


In [24]:
# inp,att = tokenize_text(repl)
# #a = inp['input_ids']
# print(inp.shape)

In [25]:
# # Test the tokenizer:
model_name = 'distilgpt2'
if random_init_wts:
    config = AutoConfig.from_pretrained(model_name, vocab_size = 50304)
    # Initialize the model with random weights
    model = AutoModelForCausalLM.from_config(config)
else:
    #model = AutoModelForCausalLM.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)


model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

    
#model = torch.compile(model)


In [26]:
tokenizer.pad_token_id

50257

### Data loaders and Dataset for batched training

In [27]:
class dataset_pyt_train(Dataset):
    def __init__(self, text_list, B = B, T = T, tokenizer = tokenizer, comp_ratio = comp_ratio,prob = .20):
        self.text_list = text_list
        #print(f"Value of B {B}")
        self.prob = prob
                                        
    def __getitem__(self, idx):
        #words_list[start_index:start_index + n]
        chunk = self.text_list[idx:idx + B*T*comp_ratio]
        #print(f"length of chunk = {len(chunk)}")
        random_num = random.random()
        #print(f"The random number generated = {random_num}")
        if random_num > self.prob:
            chunk = replace_adj_with_synonyms(chunk)
        inp,att = tokenize_text(chunk, tokenizer)
        input_id = inp.view(B,T)
        attention_mask = att.view(B,T)
        #print(f"shape of input_id = {input_id.shape}| shape of attention = {attention_mask.shape}")
        
        return input_id,attention_mask
        
        
    def __len__(self):
        #return the length of the dataframe
        num_chunks = comp_ratio*B*T
        return len(self.text_list)//num_chunks

In [28]:
class dataset_pyt_val(Dataset):
    def __init__(self, text_list, B = B, T = T, tokenizer = tokenizer, comp_ratio = comp_ratio):
        self.text_list = text_list
        #print(f"Value of B {B}")
                                                
    def __getitem__(self, idx):
        #words_list[start_index:start_index + n]
        chunk = self.text_list[idx:idx + B*T*comp_ratio]
        #print(f"length of chunk = {len()}")
        inp,att = tokenize_text(chunk, tokenizer)
        input_id = inp.view(B,T)
        attention_mask = att.view(B,T)
        #print(f"shape of input_id = {input_id.shape}| shape of attention = {attention_mask.shape}")
        
        return input_id,attention_mask
        
        
    def __len__(self):
        #return the length of the dataframe
        num_chunks = comp_ratio*B*T
        return len(self.text_list)//num_chunks

In [29]:
#train_dataset = dataset_pyt(filtered_df,tokenizer = tokenizer)
train_dataset = dataset_pyt_train(train_list)
val_dataset = dataset_pyt_val(val_list)

train_loader = DataLoader(train_dataset,batch_size = 1, shuffle = True , num_workers = 4, pin_memory = True)
val_loader = DataLoader(val_dataset,batch_size = 1, shuffle = True , num_workers = 4, pin_memory = True)


In [30]:
print(f"Tokenizer vocabulary size: {len(tokenizer)}")
print(f"EOS token ID: {tokenizer.eos_token_id}")
print(f"PAD token ID: {tokenizer.pad_token_id}")


Tokenizer vocabulary size: 50258
EOS token ID: 50256
PAD token ID: 50257


In [31]:
len(train_list)//(3*B*T)

275

In [32]:
def write_file(log_message, model_name = "GPT2" ,random_init_wts = random_init_wts ):
    current_datetime = datetime.datetime.now()
    # Extract date and time components
    current_date = str(current_datetime.date())
    log_file = model_name +'_nll_loss_adj_replacement'+'random_init_wts'+ '_'+str(random_init_wts)+'_' +current_date+'.log'
    print(f"*****LOGGING INFO IN {log_file}*********")
    filepath = os.path.join("model",log_file)
    logging.basicConfig(filename=filepath, 
                    filemode='a',  # Overwrite the log file each time
                    format='%(asctime)s - %(levelname)s - %(message)s', 
                    level=logging.DEBUG)
    logger = logging.getLogger()
    logger.info(log_message)
    
    

    
    

In [33]:
print(f"Length of the train loader is {len(train_loader)}")
print(f"Length of the val loader is {len(val_loader)}")
print(f"num_tokens= {B*T*len(train_loader)}")

Length of the train loader is 275
Length of the val loader is 287
num_tokens= 18022400


In [34]:
#emb,att,inp = train_dataset[1]

In [35]:
class check_train_metrics:
    def __init__(self, patience=25, min_delta=0 , B = T, T = T,best_loss = torch.inf,early_stop = False):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = best_loss
        self.early_stop = early_stop
        self.B = B
        self.T = T
        self.improvement = None

    def __call__(self, loss, epoch , epoch_durn, norm , current_lr, num_token):
        if self.best_loss - loss > self.min_delta:
            
            print(f"training loss has decreased---> reducing the best loss from {self.best_loss:.2f} to {loss:.2f} | throughput = {int(num_token/epoch_durn)} tokens/second | norm = {norm:.4f} | learning rate = {current_lr:.5e}")
            self.best_loss = loss
            self.counter = 0
            self.improvement = True
        else:
            self.counter += 1
            self.improvement = False
            print(f"No improvement in training  loss-->epoch= {epoch} and best loss is {self.best_loss:.2f}|current_loss = {loss}|counter = {self.counter}")
            if self.counter >= self.patience:
                self.early_stop = True


In [36]:
class check_val_metrics:
    def __init__(self, patience=25, min_delta=0, best_loss = torch.inf,early_stop = False):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = best_loss
        self.early_stop = early_stop
        

    def __call__(self, loss, epoch , model, tokenizer):
        if self.best_loss - loss > self.min_delta:
            print(f"Val loss has decreased -->reducing the global validation loss from {self.best_loss:.2f} to {loss:.2f}")
            s1 = (f"Val loss has decreased -->reducing the global validation loss from {self.best_loss:.2f} to {loss:.2f}")
            print(f" validation loss for epoch = {epoch} is {loss:.4f}")
            self.best_loss = loss
            s2 = f" validation loss for epoch = {epoch} is {loss:.4f}"
            print(f" epoch= {epoch} :  val loss is {loss:.4f} ")
            s3 = f" epoch= {epoch} :  val loss is {loss:.4f} "
            #save the model
            # Get the current date and time
            current_datetime = datetime.datetime.now()
            # Extract date and time components
            current_date = str(current_datetime.date())
            current_time = str(current_datetime.time()).split('.')[0]
            file_name = 'model'+ current_date+current_time+'.pth'
            path = os.path.join("model",file_name)
            print(f"saving the model {file_name}")
            s4 = f"saving the model {file_name}"
            #torch.save(model.state_dict(), path)
            model.save_pretrained(path)
            tokenizer.save_pretrained(path)
            log_message = s1+s2+s3+s4
            write_file(log_message)
            self.counter = 0
        else:
            self.counter += 1
            print(f"No improvement in validation loss-->epoch= {epoch} and best val loss is {self.best_loss:.2f}|current_Val loss = {loss}|counter = {self.counter}")
            if self.counter >= self.patience:
                self.early_stop = True

In [37]:
#model_output = model(input_ids = inp ,attention_mask = att, labels = inp)

In [38]:
@torch.no_grad

def eval_model(val_loader, model, epoch , device = device,tokenizer = tokenizer):
    model.eval()
    model.to(device)
    e = epoch+1
    val_loss_accum = 0.0
    print(f"inside validation data for epoch {e}")
    for ind,(input_id,attention_mask) in enumerate(val_loader):
        ids = input_id.to(device=device, non_blocking=True)
        ids = torch.squeeze(ids, dim = 0)
        att_mask = attention_mask.to(device=device, non_blocking=True)
        att_mask =  torch.squeeze(att_mask, dim = 0)
        labels = ids.clone().to(device)
        with autocast(dtype = torch.bfloat16):
            model_output = model(input_ids = ids ,attention_mask = att_mask, labels = labels)
            #print(f"model_output loss = {model_output.loss}")
            total_loss = model_output.loss  
    
    
    
        val_loss_accum+= total_loss.detach().item()
        del att_mask,labels,model_output,total_loss,ids
    return val_loss_accum        
    
    
    


In [39]:

def train_model(train_loader,val_loader,model,num_epoch = 100,device = device,tokenizer = tokenizer):
    #model.train()
    device = device
    lr_custom = 5e-5
    print(f"inside train model. Device = {device}")
    model.to(device)
    optimizer = torch.optim.AdamW(params =  model.parameters(), lr= lr_custom,fused = True ,weight_decay = .1)
      
    extra_train = .1*num_epoch
    max_train_steps = int(num_epoch +extra_train )
    import time
    from transformers import get_linear_schedule_with_warmup
    total_steps = len(train_loader) * num_epoch
    scheduler_cos = transformers.get_cosine_schedule_with_warmup( optimizer= optimizer, num_warmup_steps =int(total_steps * 0.1) ,num_training_steps= total_steps )
        
    epoch_train_log = []
    epoch_val_log = []
    validate_val_metric = check_val_metrics()
    validate_train_metric = check_train_metrics()
    for i in range (max_train_steps):
        
        epoch_start_time = time.time()
        optimizer.zero_grad(set_to_none=True)
        # we use 2 schedulers - the first LR scheduler uses a cosine decay for 100 epochs the second scheduler takes the last LR from cosine scheduler and then maintains that LR for the next 10 epochs
        if i >= num_epoch:
            optimizer_reduced_lr = torch.optim.AdamW(params =  model.parameters(), lr= current_lr ,fused = True , weight_decay=.1)
            scheduler_constant = transformers.get_constant_schedule_with_warmup( optimizer = optimizer_reduced_lr ,num_warmup_steps = 0, last_epoch = -1 )
        
        epoch_train_loss = 0       
        for ind,(input_id,attention_mask) in enumerate(train_loader):
            if ind == int(len(train_loader)/2):
                batch_time = time.time()
                duration = batch_time - epoch_start_time
                print(f"executing epoch:{i+1}, it took {duration/60} mins from beginning of epoch till batch#{ind}")
            
            ids = input_id.to(device=device, non_blocking=True)
            ids = torch.squeeze(ids, dim = 0)
            att_mask = attention_mask.to(device=device, non_blocking=True)
            att_mask =  torch.squeeze(att_mask, dim = 0)
            labels = ids.clone().to(device)
                        
            with autocast(dtype = torch.bfloat16):
                model_output = model(input_ids = ids ,attention_mask = att_mask, labels = labels)
                total_loss = model_output.loss
                if np.isnan(model_output.loss.item()):
                    print("f nan values encountered..")
                    decoded_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in ids]
                    print(f"*********$$$$$$$$$ decoded_texts = {decoded_texts}*************")
            assert not np.isnan(model_output.loss.item()), "NaN value found"
                               
            total_loss.backward()
            epoch_train_loss += total_loss.detach().item()
            norm = torch.nn.utils.clip_grad_norm(model.parameters() , 1.0)
            if i <= num_epoch:
                optimizer.step()
                scheduler_cos.step()
                optimizer.zero_grad(set_to_none=True)
            else:
                optimizer_reduced_lr.step()
                optimizer_reduced_lr.zero_grad(set_to_none=True)
                scheduler_constant.step()
                
                         
            del att_mask,labels,model_output,ids
            
        #batch processing complete 
        #print(f"batch processing complete , lambda = {lambda_val} |total_loss for batch= {total_loss}")
        
        if i <= num_epoch:
            current_lr = scheduler_cos.get_last_lr()[0]
        epoch_end_time = time.time()
        epoch_durn = (epoch_end_time - epoch_start_time)
        num_token = B*T*len(train_loader)
        epoch_train_log.append(epoch_train_loss)
        validate_train_metric(epoch_train_loss, i , epoch_durn, norm , current_lr, num_token)
        
        if validate_train_metric.improvement:
            val_loss= eval_model(val_loader, model, epoch = i, device = device,tokenizer = tokenizer)
            epoch_val_log.append(val_loss)
            validate_val_metric(val_loss, i , model, tokenizer)
            if validate_train_metric.early_stop or validate_val_metric.early_stop :
                print(f"early stopping trigerred either from training data or val data | train_counter = {validate_train_metric.counter}|val_counter = {validate_val_metric.counter}")
                break
        else:
            if validate_val_metric.early_stop:
                print(f"early stopping trigerred from validation data")
                break
              
    
    return model,epoch_train_log,epoch_val_log
        
            
            
    
    

In [ ]:
tr_model,epoch_train_log,epoch_val_log = train_model(train_loader, val_loader,model=model,tokenizer = tokenizer)

inside train model. Device = cuda:0
executing epoch:1, it took 1.4281721274058023 mins from beginning of epoch till batch#137
training loss has decreased---> reducing the best loss from inf to 2188.70 | throughput = 106162 tokens/second | norm = 1.9495 | learning rate = 5.00000e-06
inside validation data for epoch 1
Val loss has decreased -->reducing the global validation loss from inf to 1851.48
 validation loss for epoch = 0 is 1851.4790
 epoch= 0 :  val loss is 1851.4790 
saving the model model2024-07-2803:16:31.pth
[2024-07-28 03:16:33,101] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


df: /root/.triton/autotune: No such file or directory
/opt/conda/compiler_compat/ld: cannot find -laio
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible
*****LOGGING INFO IN GPT2_nll_loss_adj_replacementrandom_init_wts_True_2024-07-28.log*********
executing epoch:2, it took 1.3618555545806885 mins from beginning of epoch till batch#137
training loss has decreased---> reducing the best loss from 2188.70 to 1339.69 | throughput = 111805 tokens/second | norm = 2.4176 | learning rate = 1.00000e-05
inside validation data for epoch 2
Val loss has decreased -->reducing the globa

In [ ]:
print(epoch_train_log)


In [ ]:
# import json , os
# path_var_train_log = os.path.join(".","Tokenizers_and_loss_vals","epoch_train_plain_loss.json")
# path_var_val_log = os.path.join(".","Tokenizers_and_loss_vals","epoch_val_val_loss_.json")

# #print(path_var)
# #Write the list to a JSON file
# with open(path_var_train_log, "w") as file:
#     json.dump(epoch_train_log, file)

# with open(path_var_val_log, "w") as file:
#     json.dump(epoch_val_log, file)


In [ ]:
# with open(path_var_train_log, "r") as file:
#     train_loss = json.load(file)
# with open(path_var_val_log, "r") as file:
#     val_loss = json.load(file)

In [ ]:
x_values = range(len(epoch_train_log))
plt.plot(x_values, epoch_train_log, label='Train_loss')

In [ ]:
#epoch_val_log= [t.detach().cpu().numpy() for t in epoch_val_log]
x_values_val = range(len(epoch_val_log))
plt.plot(x_values_val, epoch_val_log, label='val_loss')

In [ ]:
min_length = min(len(epoch_train_log), len(epoch_val_log))
list1 = epoch_train_log[:min_length]
list2 = epoch_val_log[:min_length]

# Create x-axis values
x_values = range(min_length)

# Plot the lists
plt.plot(x_values, list1, label='Train_loss')
plt.plot(x_values, list2, label='Val_loss')

# Adding labels and title
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Train Loss and Val_Loss')
plt.legend()